<a href="https://colab.research.google.com/github/shshinc/HTP/blob/main/src_Jambti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 10.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import csv
import os
import cv2
import pandas as pd
import numpy as np
import torch
import multiprocessing

In [ ]:
# Opencv-Contour
def contour(image):
    area = 0.0
    cx = 0.0
    # 연산속도를 높이기위해 1차원 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 노이즈 제거를 위한 가우시안 블러 적용
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    # Canny 엣지 검출
    edges = cv2.Canny(blurred_image, threshold1=30, threshold2=100)
    # 엣지 강화
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilated_edges = cv2.dilate(edges, kernel, iterations=2)
    # Contour 검출
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Contour 내에서 가장 작은 x, y 좌표와 너비 높이를 활용
        x, y, w, h = cv2.boundingRect(contour)
        if area < w * h:
            area = w * h
            cx = x + w / 2.0
    return area, cx

# Size - house, tree
def size_ht(box_size, image_width, image_height):
    size_rate = box_size / (image_width * image_height)
    if size_rate < 0.16:
        return 'small'
    elif size_rate < 0.6:
        return 'middle'
    else:
        return 'big'

# Size - person
def size_p(box_size, image_width, image_height):
    size_rate = box_size / (image_width * image_height)
    if size_rate < 0.16:
        return 'small'
    elif size_rate < 0.4:
        return 'middle'
    else:
        return 'big'

# Loc - house, tree, person
def loc_htp(cx, image_width):
    position = image_width / 3.0
    if cx < position:
        return 'left'
    elif cx < 2 * position:
        return 'center'
    else:
        return 'right'

In [ ]:
## House
def process_house(folder_path):
    # House Model 경로
    model = YOLO('/content/drive/MyDrive/공모전/htp_Jambti/house_best.pt')

    # test_house 리스트
    house_list = []

    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_id = os.path.splitext(file_name)[0]
        # House 초기값
        house_info = {
            'id': file_id,
            'door_yn': 'n',
            'loc': None,
            'roof_yn': 'n',
            'window_cnt': 'absence',
            'size': None
            }

        # 이미지 파일 경로설정 및 로딩
        image_path = os.path.join(folder_path, file_name)
        house_origin = cv2.imread(image_path)

        # 3-Channel Grayscale 변환
        img = cv2.resize(house_origin, (640, 640))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        test_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Inference
        results = model.predict(test_image, conf = 0.3)

        ########### results -> csv ##########

        # Crop을 위한 Frame 좌표(검정 테두리)
        frame_x1, frame_x2, frame_y1, frame_y2 = 5, 615, 100, 615

        window_count = 0
        box_size = 0 # 가장 큰 박스 크기를 가진 house를 선택하기위함
        box_ok = False # House 탐지여부

        for result in results:
            # 탐지된 객체가 아무것도 없을 때 : Frame범위로 Crop
            if len(result.boxes.cls) == 0:
                crop_image = test_image[frame_y1:frame_y2, frame_x1:frame_x2]

            # House가 탐지된 경우
            elif (np.isin(0, np.array(result.boxes.cls))):
                for box in result.boxes:
                    box_ok = True
                    # 가장 큰 house를 대표 house로 선정해서 box_size, cx 구한다
                    if (box.cls == 0) and (box.xywh[0, 2].item() * box.xywh[0, 3].item()> box_size):
                        box_size = box.xywh[0, 2].item() * box.xywh[0, 3].item()
                        cx = box.xywh[0, 0]
                    elif box.cls == 1: # 지붕
                        house_info['roof_yn'] ='y'
                    elif box.cls == 2: # 창문
                        window_count += 1
                    elif box.cls == 3: # 문
                        house_info['door_yn'] = 'y'
            # House는 탐지 못했지만 하위 Object들이 탐지된 경우
            else:
                x_list =[] # Object의 x_min, x_max를 담을 리스트
                for box in result.boxes:
                    if box.cls == 1: # 지붕
                        house_info['roof_yn'] ='y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                        frame_y1 = int(round(box.xyxy[0, 1].item()))  # 지붕을 집의 꼭대기로 판단하여 y_min값 적용
                    elif box.cls == 2: # 창문
                        window_count += 1
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 3: # 문
                        house_info['door_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                # 탐지된 객체들을 모아 가장 왼쪽 좌표부터 가장 오른쪽 좌표까지 Crop
                crop_image = test_image[frame_y1:frame_y2, int(round(min(x_list))):int(round(max(x_list)))]

        # Window_cnt
        if window_count <= 2:
            house_info['window_cnt'] = '1 or 2'
        else:
            house_info['window_cnt'] = 'more than 3'

        # House가 없으면 crop한 이미지가 생김 -> contour를 통해 box_size, cx 예측
        if box_ok is False:
            box_size, cx = contour(crop_image)

        # 원본 사이즈 비율로 조정
        crop_y = house_origin.shape[0] * test_image.shape[0] / house_origin.shape[1]
        box_size = box_size * crop_y / test_image.shape[0]

        # House를 맞추기 위한 최적의 상수
        h = 1.28
        # 원본 비율을 기준으로 size, loc 범주화
        house_info['size'], house_info['loc'] = size_ht(box_size * h, test_image.shape[1], crop_y), loc_htp(cx, test_image.shape[1]) # size = crop_size * h

        # 파일 정보 리스트에 저장
        house_list.append(house_info)

    # CSV 파일로 데이터 저장 : output폴더 경로
    with open('/content/drive/MyDrive/공모전/htp_Jambti/output/test_house.csv', 'w', newline='') as csvfile:
        fieldnames = house_list[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in house_list:
            writer.writerow(row)
    print("test_house.csv 생성완료")

In [ ]:
## Tree
def process_tree(folder_path):
    # Tree Model 경로
    model = YOLO('/content/drive/MyDrive/공모전/htp_Jambti/tree_best.pt') # 경로 수정

    # test_tree 리스트
    tree_list = []

    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_id = os.path.splitext(file_name)[0]
        # Tree 초기값
        tree_info = {
            'id': file_id,
            'branch_yn': 'n',
            'root_yn': 'n',
            'crown_yn': 'n',
            'fruit_yn': 'n',
            'gnarl_yn': 'n',
            'loc':None,
            'size':None
            }

        # 이미지 파일 경로설정 및 로딩
        image_path = os.path.join(folder_path, file_name)
        tree_origin = cv2.imread(image_path)

        # 3-Channel Grayscale 변환
        img = cv2.resize(tree_origin, (640, 640))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        test_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Inference
        results = model.predict(test_image, conf = 0.5)

        ########### results -> csv ##########

        # Crop을 위한 Frame 좌표(검정 테두리)
        frame_x1, frame_x2, frame_y1, frame_y2 = 5, 615, 100, 615

        box_size = 0 # 가장 큰 박스 크기를 가진 tree를 선택하기위함
        box_ok = False # Tree 탐지여부

        for result in results:
            # 탐지된 객체가 아무것도 없을 때 : Frame범위로 Crop
            if len(result.boxes.cls) == 0:
                crop_image = test_image[frame_y1:frame_y2, frame_x1:frame_x2]

            # Tree가 탐지된 경우
            elif (np.isin(0, np.array(result.boxes.cls))):
                for box in result.boxes:
                    box_ok = True
                    # 가장 큰 tree를 대표 tree로 선정해서 box_size, cx 구한다
                    if (box.cls == 0) and (box.xywh[0, 2].item() * box.xywh[0, 3].item()> box_size):
                        box_size = box.xywh[0, 2].item() * box.xywh[0, 3].item()
                        cx = box.xywh[0, 0]
                    elif box.cls == 1: # 옹, 상처
                        tree_info['gnarl_yn'] ='y'
                    elif box.cls == 2: # 수관
                        tree_info['crown_yn'] ='y'
                    elif box.cls == 3: # 가지
                        tree_info['branch_yn'] = 'y'
                    elif box.cls == 4: # 뿌리
                        tree_info['root_yn'] = 'y'
                    elif box.cls == 5: # 열매
                        tree_info['fruit_yn'] = 'y'
            # Tree는 탐지 못했지만 하위 Object들이 탐지된 경우
            else:
                x_list =[]
                for box in result.boxes:
                    if box.cls == 1: # 옹이나 상처
                        tree_info['gnarl_yn'] ='y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 2: # 수관
                        tree_info['crown_yn'] ='y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 3: # 가지
                        tree_info['branch_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 4: # 뿌리
                        tree_info['root_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                        frame_y2 = int(round(box.xyxy[0, 3].item())) # 뿌리를 나무의 최저점로 판단하여 y_max값 적용
                    elif box.cls == 5: # 열매
                        tree_info['fruit_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                # 탐지된 객체들을 모아 가장 왼쪽 좌표부터 가장 오른쪽 좌표까지 Crop
                crop_image = test_image[frame_y1:frame_y2, int(round(min(x_list))):int(round(max(x_list)))]

        # Tree가 없으면 crop한 이미지가 생김 -> contour를 통해 box_size, cx 예측
        if box_ok is False:
            box_size, cx = contour(crop_image)

        # 원본 사이즈 비율로 조정
        crop_y = tree_origin.shape[0] * test_image.shape[0] / tree_origin.shape[1]
        box_size = box_size * crop_y / test_image.shape[0]

        # Tree를 맞추기 위한 최적의 상수
        t = 1.4
        # 원본 비율을 기준으로 size, loc 범주화
        tree_info['size'], tree_info['loc'] = size_ht(box_size * t, test_image.shape[1], crop_y), loc_htp(cx, test_image.shape[1]) # size = crop_size * t

        # 파일 정보 리스트에 저장
        tree_list.append(tree_info)

    # CSV 파일로 데이터 저장 : output폴더 경로
    with open('/content/drive/MyDrive/공모전/htp_Jambti/output/test_tree.csv', 'w', newline='') as csvfile:
        fieldnames = tree_list[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in tree_list:
            writer.writerow(row)
    print("test_tree.csv 생성완료")

In [ ]:
## Person
def process_person(folder_path):
    # Person Model 경로
    model = YOLO('/content/drive/MyDrive/공모전/htp_Jambti/person_best.pt')

    # test_person 리스트
    person_list = []

    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_id = os.path.splitext(file_name)[0]
        # Person 초기값
        person_info = {
            'id': file_id,
            'eye_yn': 'n',
            'leg_yn': 'n',
            'loc':None,
            'mouth_yn': 'n',
            'size':None,
            'arm_yn': 'n',
            }

        # 이미지 파일 경로설정 및 로딩
        image_path = os.path.join(folder_path, file_name)
        person_origin = cv2.imread(image_path)

        # 3-Channel Grayscale 변환
        img = cv2.resize(person_origin, (640, 640))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        test_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Inference
        results = model.predict(test_image, conf = 0.2)

        ########### results -> csv ##########

        # Crop을 위한 Frame 좌표(검정 테두리)
        frame_x1, frame_x2, frame_y1, frame_y2 = 5, 615, 100, 615
        box_size = 0 # 가장 큰 박스 크기를 가진 Person를 선택하기위함
        box_ok = False # Person 탐지여부

        for result in results:
            # 탐지된 객체가 아무것도 없을 때 : Frame범위로 Crop
            if len(result.boxes.cls) == 0:
                crop_image = test_image[frame_y1:frame_y2, frame_x1:frame_x2] # image crop (이미지 테두리 기준)

            # Person가 탐지된 경우
            elif (np.isin(0, np.array(result.boxes.cls))):
                for box in result.boxes:
                    box_ok = True
                    # 가장 큰 person를 대표 person으로 선정해서 box_size, cx 구한다
                    if (box.cls == 0) and (box.xywh[0, 2].item() * box.xywh[0, 3].item()> box_size):
                        box_size = box.xywh[0, 2].item() * box.xywh[0, 3].item()
                        cx = box.xywh[0, 0]
                    elif box.cls == 1: # 눈
                        person_info['eye_yn'] ='y'
                    elif box.cls == 2: # 입
                        person_info['mouth_yn'] ='y'
                    elif box.cls == 3: # 팔
                        person_info['arm_yn'] = 'y'
                    elif box.cls == 4: # 다리
                        person_info['leg_yn'] = 'y'
            # person은 탐지 못했지만 하위 Object들이 탐지된 경우
            else:
                x_list =[]# Object의 x_min, x_max를 담을 리스트
                for box in result.boxes:
                    if box.cls == 1: # 눈
                        person_info['eye_yn'] ='y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 2: # 입
                        person_info['mouth_yn'] ='y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 3: # 팔
                        person_info['arm_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                    elif box.cls == 4: # 다리
                        person_info['leg_yn'] = 'y'
                        x_list.append(box.xyxy[0, 0].item())
                        x_list.append(box.xyxy[0, 2].item())
                        frame_y2 = int(round(box.xyxy[0, 3].item())) # 다리를 사람의 최저점으로 판단하여 y_max값 적용
                # 탐지된 객체들을 모아 가장 왼쪽 좌표부터 가장 오른쪽 좌표까지 Crop
                crop_image = test_image[frame_y1:frame_y2, int(round(min(x_list))):int(round(max(x_list)))]


        # Person이 없으면 crop한 이미지가 생김 -> contour를 통해 box_size, cx 예측
        if box_ok == False:
            box_size, cx = contour(crop_image)

        # 원본 사이즈 비율로 조정
        crop_y = person_origin.shape[0] * test_image.shape[0] / person_origin.shape[1]
        box_size = box_size * crop_y / test_image.shape[0]

        # Person 맞추기 위한 최적의 상수
        p =  1.5
        # 원본 비율을 기준으로 size, loc 범주화
        person_info['size'], person_info['loc'] = size_p(box_size * p, test_image.shape[1], crop_y), loc_htp(cx, test_image.shape[1])

        # 파일 정보 리스트에 저장
        person_list.append(person_info)

    # CSV 파일로 데이터 저장 : output폴더 경로
    with open('/content/drive/MyDrive/공모전/htp_Jambti/output/test_person.csv', 'w', newline='') as csvfile:
        fieldnames = person_list[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in person_list:
            writer.writerow(row)
    print("test_person 생성")

In [ ]:
# Input 폴더 경로
input_folder = '/content/drive/MyDrive/공모전/htp_Jambti/input'

if __name__ == "__main__":
    # input_folder 내의 모든 하위 폴더 목록 가져오기
    subfolders = [os.path.join(input_folder, folder) for folder in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, folder))]

    # 각 폴더에 대한 멀티프로세스 작업 시작
    processes = []
    for folder_path in subfolders:
        folder_name = os.path.basename(folder_path)
        if folder_name == 'house':
            process = multiprocessing.Process(target=process_house, args=(folder_path,))
        elif folder_name == 'tree':
            process = multiprocessing.Process(target=process_tree, args=(folder_path,))
        elif folder_name == 'person':
            process = multiprocessing.Process(target=process_person, args=(folder_path,))
        processes.append(process)
        process.start()

    # 모든 멀티프로세스가 종료될 때까지 대기
    for process in processes:
        process.join()

    print("모든 작업이 완료됨")





0: 640x640 1 tree, 824.1ms
Speed: 16.3ms preprocess, 824.1ms inference, 26.4ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 1 person, 2 eyes, 1 mouth, 2 arms, 2 legs, 902.5ms
0: 640x640 1 house, 1 roof, 1 window, 1 door, 931.4ms
Speed: 10.4ms preprocess, 902.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)
Speed: 8.2ms preprocess, 931.4ms inference, 6.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 house, 1 roof, 3 windows, 1 door, 731.8ms
Speed: 11.1ms preprocess, 731.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 1 tree, 1 crown, 541.8ms
Speed: 8.5ms preprocess, 541.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 1 person, 2 eyes, 1 mouth, 2 arms, 2 legs, 469.8ms
Speed: 4.7ms preprocess, 469.8ms inference, 13.4ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 1 house, 1 roof, 3 windows, 1 door, 670.7ms
Speed: 3.7ms preprocess, 670.7ms inference, 7.5

test_tree.csv 생성완료



0: 640x640 1 house, 1 roof, 2 windows, 1 door, 315.1ms
Speed: 2.9ms preprocess, 315.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 2 eyes, 2 mouths, 2 arms, 1 leg, 217.7ms
Speed: 2.8ms preprocess, 217.7ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 1 house, 2 roofs, 6 windows, 1 door, 336.2ms
Speed: 2.8ms preprocess, 336.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 (no detections), 282.3ms
Speed: 3.1ms preprocess, 282.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 house, 1 roof, 1 window, 1 door, 318.7ms
Speed: 3.0ms preprocess, 318.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 2 eyes, 1 mouth, 2 legs, 234.9ms
Speed: 2.9ms preprocess, 234.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 1 house, 1 roof, 2 windows, 1 door, 397.2ms
Speed: 3.6ms preprocess, 397.2ms

test_person 생성


0: 640x640 1 house, 1 roof, 3 windows, 1 door, 270.2ms
Speed: 3.5ms preprocess, 270.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 house, 1 roof, 4 windows, 1 door, 219.3ms
Speed: 3.1ms preprocess, 219.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 house, 1 roof, 3 windows, 1 door, 203.7ms
Speed: 2.9ms preprocess, 203.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 house, 1 roof, 5 windows, 2 doors, 213.0ms
Speed: 3.5ms preprocess, 213.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


test_house.csv 생성완료
모든 작업이 완료됨
